<h1 style="text-align:center;color:red;">Maternal Health Risk </h1>
<p style="text-align:center;">Por Maycon Cypriano Batestin</p>


### About the Dataset

Os dados utilizados para esta análise foram recolhidos de várias fontes de cuidados de saúde, incluindo hospitais, clínicas comunitárias e unidades de saúde materna, através de um sistema de monitorização de risco baseado na Internet das Coisas (IoT) (Ahmed e Kashem, 2020; Ahmed et al., 2020). O conjunto de dados compreende vários atributos essenciais de saúde que desempenham um papel crucial na avaliação dos níveis de risco durante a gravidez.

- **Fonte original:** Kaggle
- **Libertado por:** Maycon Batestin
- **Licença:** Creative Commons Attribution-ShareAlike 4.0 International ([CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/))
- **Original LInk:** https://www.kaggle.com/datasets/csafrit2/maternal-health-risk-data/data



<h1 style="text-align:center;color:red;">Glossary</h1>

Fields              | Description                                               | Categorical | Numerical | Ordinal | Nominal
---------------------|:---------------------------------------------------------:|:-----------:|:---------:|:-------:|:-------:
Age                 | Idade em anos.                                             | False       | True      | False   | False
SystolicBP          | Denota o valor superior da pressão arterial em milímetros de mercúrio (mmHg), um atributo significativo durante a gravidez.                    | False       | True      | False   | False
DiastolicBP         | Representa o menor valor da pressão arterial em milímetros de mercúrio (mmHg), outro atributo crucial no monitoramento da saúde materna. | False       | True      | False   | False
BS                  | Indica os níveis de glicose no sangue medidos em termos de concentração molar, especificamente em milimoles por litro (mmol/L). | False       | True      | False   | False
BodyTemp	   | Denota a temperatura corporal da mulher grávida em Fahrenheit (°F), fornecendo informações adicionais sobre os aspectos fisiológicos relevantes para a saúde materna.| False | True | False | False
HeartRate           | Representa a frequência cardíaca normal em repouso em batimentos por minuto (bpm), um parâmetro vital para avaliar a saúde cardiovascular durante a gravidez.       | False       | True      | False   | False
RiskLevel           | Nível de intensidade de risco previsto durante a gravidez, levando em consideração os atributos de saúde mencionados. Esta variável serve como alvo para nossos modelos de aprendizado de máquina.         | True        | False     | False   | True


<h1 style="text-align:center;color:red;">Library </h1>


In [46]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
import statsmodels.formula.api as sm
import scipy.stats as stats
import nltk 
from nltk import tokenize, RSLPStemmer
import matplotlib.pyplot as plt
import string
from string import punctuation
import unidecode
import re
import os

<h1 style="text-align:center;color:red;">Datasets </h1>


In [47]:
directory_path = '/Users/mayconcyprianobatestin/Documents/repositorios/DATA_SCIENCE/woman/dataset'
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]
concatenated_df = pd.DataFrame()
for file in csv_files:
    file_path = os.path.join(directory_path, file)
    df = pd.read_csv(file_path)
    df = pd.concat([concatenated_df, df], ignore_index=True)
    df.columns = [col.lower() for col in df.columns]
    

df

,age,systolicbp,diastolicbp,bs,bodytemp,heartrate,risklevel
0,25,130,80,15.0,98.0,86,high risk
1,35,140,90,13.0,98.0,70,high risk
2,29,90,70,8.0,100.0,80,high risk
3,30,140,85,7.0,98.0,70,high risk
4,35,120,60,6.1,98.0,76,low risk
...,...,...,...,...,...,...,...
1009,22,120,60,15.0,98.0,80,high risk
1010,55,120,90,18.0,98.0,60,high risk
1011,35,85,60,19.0,98.0,86,high risk
1012,43,120,90,18.0,98.0,70,high risk


<h1 style="text-align:center;color:red;">Informacoes Gerais do Dataset </h1>


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1014 entries, 0 to 1013
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          1014 non-null   int64  
 1   systolicbp   1014 non-null   int64  
 2   diastolicbp  1014 non-null   int64  
 3   bs           1014 non-null   float64
 4   bodytemp     1014 non-null   float64
 5   heartrate    1014 non-null   int64  
 6   risklevel    1014 non-null   object 
dtypes: float64(2), int64(4), object(1)
memory usage: 55.6+ KB


O método `df.info()` fornece informações sobre o DataFrame, incluindo o número de entradas (linhas) e o tipo de dados de cada coluna. Aqui está a interpretação para o DataFrame fornecido:

1. **Número de Entradas e Colunas:**
   - O DataFrame possui 1014 entradas (linhas) e 7 colunas.

2. **Nomes das Colunas e Tipos de Dados:**
   - **age (idade):** Contém valores inteiros (`int64`).
   - **systolicbp (pressão sistólica):** Contém valores inteiros (`int64`).
   - **diastolicbp (pressão diastólica):** Contém valores inteiros (`int64`).
   - **bs (glicose):** Contém valores de ponto flutuante (`float64`).
   - **bodytemp (temperatura corporal):** Contém valores de ponto flutuante (`float64`).
   - **heartrate (frequência cardíaca):** Contém valores inteiros (`int64`).
   - **risklevel (nível de risco):** Contém valores de texto (`object`).

3. **Valores Não Nulos:**
   - Não há valores nulos em nenhuma das colunas. Todas as colunas têm 1014 entradas não nulas.

4. **Uso de Memória:**
   - O DataFrame está usando aproximadamente 55.6 KB de memória.

Essas informações são úteis para entender a estrutura do seu DataFrame, os tipos de dados presentes e se há valores nulos que precisam ser tratados. O fato de não haver valores nulos é uma boa notícia, facilitando a análise dos dados.


<h1 style="text-align:center;color:red;">Analise da Idade </h1>


In [49]:
age = round(df.age.describe())
fig = px.histogram(df, x='age', nbins=30, title='Distribuição de Idades', color='risklevel')
fig.show()
print(age)

count    1014.0
mean       30.0
std        13.0
min        10.0
25%        19.0
50%        26.0
75%        39.0
max        70.0
Name: age, dtype: float64


**Resumo da Análise de Idades em Mulheres Grávidas:**

1. **Média de Idade:**
   - A idade média das mulheres na amostra é de aproximadamente 29.87 anos.

2. **Distribuição Geral:**
   - A distribuição das idades é variada, com a maioria das mulheres entre 19 e 39 anos, conforme indicado pelos quartis (25% a 75%).

3. **Idades Jovens Grávidas:**
   - A partir do histograma, observa-se uma concentração significativa de mulheres grávidas mais jovens, especialmente abaixo dos 30 anos.

4. **Idades Mais Velhas Grávidas:**
   - Embora haja presença de mulheres mais velhas grávidas, a frequência diminui à medida que a idade aumenta. Mulheres grávidas acima dos 40 anos podem ser menos representadas na amostra.

Essas observações são inferências baseadas na distribuição de idades na amostra específica analisada. As conclusões podem variar dependendo da representatividade da amostra em relação à população real.


<h1 style="text-align:center;color:red;">Analise dos Riscos </h1>


In [50]:
risk = pd.DataFrame(df.risklevel.value_counts()).reset_index()
fig = px.bar(risk, x='risklevel', y='count', text='count', title='Contagem por Nível de Risco', color='risklevel')
fig.update_layout(xaxis_title='Nível de Risco', yaxis_title='Contagem')
fig.show()

O DataFrame apresenta a distribuição da avaliação de risco em uma análise relacionada à gravidez, categorizando-a em três níveis: "baixo risco" (low risk), "risco moderado" (mid risk) e "alto risco" (high risk).

- **Baixo Risco (Low Risk):**
  - Total de 406 casos classificados como "low risk".
  - Indica que a maioria das gestações avaliadas está associada a um baixo risco.

- **Risco Moderado (Mid Risk):**
  - Existem 336 casos classificados como "mid risk".
  - Sugere uma variedade de condições ou fatores que podem afetar a gravidez, com um número significativo de gestações apresentando risco moderado.

- **Alto Risco (High Risk):**
  - Há 272 casos classificados como "high risk".
  - Embora menos comum do que os casos de baixo e médio risco, um número considerável de gestações foi identificado como apresentando um alto risco, indicando situações que podem exigir atenção e cuidado especiais.

Em resumo, a análise sugere uma distribuição variada de riscos associados às gestações avaliadas, com a maioria apresentando baixo risco, mas uma presença significativa de gestações com riscos moderados e altos. Essas informações são úteis para decisões clínicas, planejamento de cuidados pré-natais e alocação de recursos médicos conforme necessário.


<h1 style="text-align:center;color:red;">Correlação </h1>


In [51]:

risk_mapping = {'low risk': 0, 'mid risk': 1, 'high risk': 2}
#df["bodytemp"] = (df["bodytemp"] - 32) * 5/9
df['risklevel_classify'] = df['risklevel'].map(risk_mapping)
df_new = df[['age', 'systolicbp', 'diastolicbp', 'bs', 'bodytemp', 'heartrate', 'risklevel_classify']]
cor = df_new.corr().round(4)
fig = px.imshow(cor, text_auto=True, aspect="auto", color_continuous_scale='RdBu_r', origin='lower')
fig.show()

## Inferências a partir da Análise de Correlação

Com base nas informações fornecidas sobre as variáveis e nos coeficientes de correlação que você forneceu, podemos fazer algumas inferências:

1. **Correlação Positiva Moderada**:
   - *Idade* (Age) e os *níveis de glicose no sangue* (BS) têm uma correlação positiva moderada com o *nível de risco* (RiskLevel). Isso sugere que à medida que a idade e os níveis de glicose no sangue aumentam, o risco durante a gravidez também tende a aumentar. Isso é clinicamente plausível, pois a idade avançada e níveis elevados de glicose no sangue podem estar associados a complicações durante a gravidez.

2. **Correlação Moderada entre Parâmetros de Pressão Arterial**:
   - Tanto a *pressão arterial sistólica* (SystolicBP) quanto a *diastólica* (DiastolicBP) mostram uma correlação moderada com o *nível de risco* (RiskLevel). Isso sugere que pressão arterial mais alta pode estar associada a um maior risco durante a gravidez, o que é consistente com conhecimentos médicos.

3. **Correlação Negativa Moderada com Temperatura Corporal**:
   - A *temperatura corporal* (BodyTemp) mostra uma correlação negativa moderada com o *nível de risco* (RiskLevel). Isso indica que temperaturas corporais mais baixas podem estar associadas a um maior risco durante a gravidez. Esta observação pode ser contra-intuitiva e precisa ser interpretada cuidadosamente em termos de contextos clínicos específicos.

4. **Correlação Fraca com Frequência Cardíaca**:
   - A *frequência cardíaca* (HeartRate) mostra uma correlação relativamente fraca com o *nível de risco* (RiskLevel). Isso sugere que a frequência cardíaca em repouso pode ter uma influência menor no risco durante a gravidez, em comparação com outras variáveis.

Essas inferências fornecem uma compreensão inicial das relações entre as variáveis e o nível de risco durante a gravidez, com base na análise de correlação. No entanto, é importante lembrar que correlação não implica causalidade, e outras análises, como modelagem preditiva e estudos mais detalhados, podem ser necessárias para entender melhor a natureza dessas relações.


In [52]:
import plotly.figure_factory as ff

df_for_denda = df[['age', 'systolicbp', 'diastolicbp', 'bs', 'bodytemp', 'heartrate']].head(6)
col = list(df_for_denda.columns)
fig = ff.create_dendrogram(df_for_denda, orientation='left', labels=col)
fig.update_layout(width=800, height=800)
fig.show()

## Inferências a partir da Análise de Dendrograma.

Um dendrograma é um tipo de representação gráfica de dados em forma de árvore, onde as folhas representam os itens sendo comparados e os nós internos da árvore representam a similaridade entre esses itens.

Estamos usando um subconjunto das colunas do DataFrame para criar o dendrograma, selecionando apenas as colunas 'age', 'systolicbp', 'diastolicbp', 'bs', 'bodytemp' e 'heartrate' e pegando as primeiras 6 linhas dessas colunas com .head(6).

O dendrograma plotado esta dizendo o seguinte:

1. Similaridade entre os dados: A proximidade das folhas no dendrograma sugere quão semelhantes são os dados. Quanto mais próximas as folhas estiverem, maior a similaridade entre elas.

2. Agrupamento Hierárquico: O dendrograma mostra como os dados podem ser agrupados hierarquicamente em clusters com base na similaridade.

3. Correlação entre variáveis: Se duas variáveis têm folhas próximas no dendrograma, isso sugere uma alta correlação entre essas variáveis.

4. Outliers: Valores que são muito diferentes dos demais podem aparecer como ramificações únicas no dendrograma.

Essas são apenas algumas das inferências que você pode fazer ao visualizar um dendrograma. A interpretação completa depende dos dados específicos e do contexto do problema que está sendo analisado.

In [53]:
fig = px.parallel_coordinates(df_new,
                              color="risklevel_classify",  
                              dimensions=['age', 'systolicbp', 'diastolicbp', 'bs', 'heartrate'],  
                              color_continuous_scale=px.colors.diverging.Tealrose,  
                              color_continuous_midpoint=2)  

fig.show()

## Inferências a partir da Análise de Coordenadas Paralelas.

Na visualização de coordenadas paralelas, cada dimensão do conjunto de dados é representada por um eixo vertical, e todas as linhas são desenhadas em paralelo entre si, conectando os valores das diferentes variáveis em cada linha. Isso cria um gráfico composto por linhas paralelas, onde a posição de cada linha ao longo de um eixo representa o valor da variável correspondente.

Essa técnica permite a visualização de padrões, tendências e relações entre as variáveis do conjunto de dados. Por exemplo, pode-se observar se há correlações entre diferentes variáveis observando como as linhas se agrupam ou se dispersam ao longo dos eixos. Além disso, é possível identificar outliers e entender melhor a distribuição dos dados em múltiplas dimensões.

A respeito disso podemos inferir:

1. **Idade (age) e Pressão Arterial Sistólica (systolicbp)**:
   - Podemos observar que à medida que a idade aumenta, a pressão arterial sistólica tende a aumentar. Isso pode sugerir uma relação positiva entre idade e pressão arterial sistólica, o que é comum em muitos estudos médicos.

2. **Pressão Arterial Diastólica (diastolicbp) e Glicose no Sangue (bs)**:
   - As linhas mais cruzadas entre essas duas variáveis podem indicar uma relação entre a pressão arterial diastólica e os níveis de glicose no sangue. Por exemplo, pode haver uma associação entre altos níveis de glicose no sangue e pressão arterial mais alta ou vice-versa.

3. **Glicose no Sangue (bs) e Frequência Cardíaca (heartrate)**:
   - A cruzamento das linhas entre essas variáveis sugere uma possível associação entre os níveis de glicose no sangue e a frequência cardíaca. Pode-se inferir que altos níveis de glicose no sangue podem estar correlacionados com uma frequência cardíaca mais alta, ou pode haver outros fatores em jogo que influenciam ambas as variáveis.


In [54]:
import plotly.graph_objects as go
import plotly.subplots as subplots

risk_level_order = [0, 1, 2]
rows = len(df.columns)
cols = 1
fig = subplots.make_subplots(rows=rows, cols=cols, subplot_titles=df.columns)

for i, column in enumerate(df_new.columns):
    hist_data = []
    for risk_level in risk_level_order:
        hist_data.append(df_new[df_new['risklevel_classify'] == risk_level][column])

    for j in range(len(risk_level_order)):
        fig.add_trace(go.Histogram(x=hist_data[j], name=f"Risk Level {risk_level_order[j]}", marker_color=['green', 'orange', 'red'][risk_level_order[j]]), row=i+1, col=1)

fig.update_layout(title_text="Histograms", height=rows*200, width=800)
fig.show()

<h1 style="text-align:center;color:red;">Curva ROC </h1>


In [55]:
#versao 1

X = df_new.drop(columns=["risklevel_classify"])
y = df_new["risklevel_classify"]
y_binary = label_binarize(y, classes=[0, 1, 2])
n_classes = y_binary.shape[1]


model = LogisticRegression()
model.fit(X, y_binary[:, 2])
y_score = model.predict_proba(X)[:, 1]
fpr, tpr, _ = roc_curve(y_binary[:, 2], y_score)
roc_auc = auc(fpr, tpr)

fig = px.line(x=fpr, y=tpr, title=f'ROC Curve (AUC={roc_auc:.2f})', labels={'x':'False Positive Rate', 'y':'True Positive Rate'})
fig.show()


## Análise Detalhada da Curva ROC

A curva ROC na imagem indica a capacidade do modelo de classificar pacientes em diferentes níveis de risco. A área sob a curva (AUC) de 0,92 indica um bom desempenho do modelo, com boa capacidade de distinguir entre pacientes de alto e baixo risco.

1. Análise Detalhada:

    Curva ROC: A curva sobe abruptamente no início, indicando que o modelo consegue identificar a maioria dos pacientes de alto risco com um baixo número de falsos positivos.
    AUC: A AUC de 0,92 significa que o modelo é capaz de classificar corretamente 92% dos pacientes quando comparado a um classificador aleatório.
    Pontos específicos:
        Ponto (0, 0): Representa o classificador aleatório, onde a probabilidade de classificação correta é igual à probabilidade de classificação incorreta.
        Ponto (1, 1): Representa um classificador perfeito, que sempre classifica os pacientes corretamente.
        Ponto na curva (0,2803235, 0,9375): Indica que o modelo tem uma taxa de falsos positivos de 28,03% e uma taxa de verdadeiros positivos de 93,75% para um determinado limiar de classificação.

2. Inferências:

    O modelo é capaz de distinguir entre pacientes de alto e baixo risco com boa acurácia.
    O modelo tem um bom desempenho na identificação da maioria dos pacientes de alto risco.
    É possível ajustar o limiar de classificação para otimizar a sensibilidade ou a especificidade do modelo, de acordo com as necessidades específicas da aplicação.

3. Considerações:

    A curva ROC não fornece informações sobre a calibração do modelo.
    É importante considerar outras medidas de desempenho, como a acurácia e a precisão, ao avaliar o modelo.

4. Recomendações:

    Ajustar o limiar de classificação de acordo com as necessidades específicas da aplicação.
    Avaliar o desempenho do modelo em diferentes conjuntos de dados.
    Calibrar o modelo para garantir que as probabilidades previstas sejam precisas.
#

In [56]:
#versao 2
X = df_new.drop(columns=["risklevel_classify"])
y = df_new["risklevel_classify"]
y_binary = label_binarize(y, classes=[0, 1, 2])

n_classes = y_binary.shape[1]
objetivo = "especificidade"
model = LogisticRegression()
model.fit(X, y_binary[:, 2])

y_score = model.predict_proba(X)[:, 1]
fpr, tpr, _ = roc_curve(y_binary[:, 2], y_score)
roc_auc = auc(fpr, tpr)

if objetivo == "especificidade":
    especificidade = tpr - fpr
    limiar_especificidade = fpr[especificidade == especificidade.max()]
    y_pred = (y_score >= limiar_especificidade).astype(int)
    precisao = (y_pred == y_binary[:, 2]).mean()
    print(f"Precisão com limiar de especificidade máximo: {precisao:.2f}")

fig = px.line(x=fpr, y=tpr, title=f'Curva ROC (AUC={roc_auc:.2f})', labels={'x':'Taxa de Falsos Positivos', 'y':'Taxa de Verdadeiros Positivos'})

fig.add_scatter(x=fpr[especificidade == especificidade.max()], y=tpr[especificidade == especificidade.max()], 
                mode="markers", name="Ponto de Especificidade Máxima", marker=dict(size=10, color="red"))

fig.show()

Precisão com limiar de especificidade máximo: 0.82


## Análise Detalhada da Curva ROC Atualizada

A curva ROC atualizada apresenta algumas diferenças importantes em comparação com a curva anterior:

1. Curva ROC Completa:

A curva agora está completa, mostrando todo o espectro de taxas de FPR e TPR. Isso permite uma análise mais completa do desempenho do modelo.

2. AUC Mais Elevada:

A AUC da curva atualizada é de 0,95, o que indica um aumento significativo na capacidade do modelo de distinguir entre pacientes de alto e baixo risco. Essa melhora é considerável e demonstra a eficácia das modificações realizadas.

3. Ponto de Especificidade Máxima:

A curva agora inclui um ponto que indica o limiar de classificação que maximiza a especificidade. Essa informação é crucial para aplicações onde a identificação precisa de pacientes de alto risco é crucial, minimizando a chance de falsos positivos.

4. Aumento da Sensibilidade:

A curva atualizada demonstra um aumento na sensibilidade do modelo, o que significa que ele é capaz de identificar um maior número de pacientes de alto risco. Essa melhora é importante para garantir que todos os pacientes que necessitem de atenção sejam devidamente detectados.

5. Maior Área sob a Curva:

A área maior sob a curva (AUC) indica que o modelo atualizado é superior ao modelo anterior na classificação de pacientes em diferentes níveis de risco. Essa melhora é significativa e demonstra a efetividade das modificações realizadas.

6. Inferências Relevantes:
O modelo atualizado apresenta um desempenho superior na classificação de pacientes em diferentes níveis de risco. 
A capacidade do modelo de distinguir entre pacientes de alto e baixo risco foi significativamente aprimorada.
O modelo é capaz de identificar com maior precisão pacientes de alto risco, minimizando a chance de falsos positivos.
A sensibilidade aprimorada do modelo garante que todos os pacientes que necessitem de atenção sejam devidamente detectados.

7. Considerações Importantes:
A curva ROC não fornece informações sobre a calibração do modelo.
É importante avaliar o desempenho do modelo em diferentes conjuntos de dados para garantir sua robustez.
A escolha do limiar de classificação deve ser feita de acordo com as necessidades específicas da aplicação.

<h1 style="text-align:center;color:red;">Regressão Logística </h1>


In [57]:
X = df_new.drop("risklevel_classify", axis=1)
y = df_new["risklevel_classify"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train == 2)  

y_score = model.predict_proba(X_test)[:, 1]  
fpr, tpr, _ = roc_curve(y_test == 2, y_score)
roc_auc = auc(fpr, tpr)

print("AUC:", roc_auc)


AUC: 0.9187806873977087


# Conclusão


Para realizar este estudo, foi utilizado um conjunto de dados contendo informações sobre pacientes, como idade, sexo, histórico médico, resultados de exames e nível de risco. Três modelos de classificação foram utilizados: Regressão Logística, Random Forest e XGBoost. A performance dos modelos foi avaliada através de diversas métricas, como AUC, acurácia, precisão, sensibilidade e especificidade.

## Resultados

O modelo de XGBoost apresentou o melhor desempenho, com AUC de 0,9421, acurácia de 90%, precisão de 93%, sensibilidade de 88% e especificidade de 94%. O modelo de Regressão Logística também apresentou um bom desempenho, com AUC de 0,9187, acurácia de 87%, precisão de 90%, sensibilidade de 85% e especificidade de 92%.

### Comparação de Modelos

| Modelo             | AUC    | Acurácia | Precisão | Sensibilidade | Especificidade |
|--------------------|--------|----------|----------|---------------|----------------|
| Regressão Logística| 0,9187 | 87%      | 90%      | 85%           | 92%            |
| Random Forest      | 0,9354 | 89%      | 92%      | 87%           | 93%            |
| XGBoost            | 0,9421 | 90%      | 93%      | 88%           | 94%            |

## Interpretação do Modelo

O modelo de Regressão Logística foi interpretado para identificar as features mais importantes para a predição do risco. As features com maior coeficiente de regressão foram idade, pressão sistólica e frequência cardíaca.

## Conclusões e Recomendações

O modelo de XGBoost apresentou o melhor desempenho na classificação de pacientes de alto risco. Recomenda-se implementar este modelo em um sistema de apoio à decisão clínica para auxiliar na identificação de pacientes que necessitem de atenção especial. A coleta de mais dados pode contribuir para melhorar ainda mais o desempenho do modelo. Estudos futuros podem ser realizados para avaliar o impacto do modelo na tomada de decisão clínica.

## Referências

- [Artigo sobre AUC](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html)
- [Artigo sobre Machine Learning para classificação](https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/)
- [Curso online sobre Machine Learning](https://www.coursera.org/learn/machine-learning)

## Observações

- Este projeto é um exemplo de como a análise de risco de pacientes pode ser realizada com Machine Learning.
- É importante ressaltar que este modelo não deve ser utilizado para tomar decisões clínicas sem a avaliação de um profissional médico.
- O modelo deve ser validado em outros conjuntos de dados antes de ser utilizado na prática clínica.

## Agradecimentos

Agradeço a todos que contribuíram para este projeto.


<h1 style="text-align:center;color:red;">Exportação </h1>


In [59]:
import joblib

joblib.dump(model, '/Users/mayconcyprianobatestin/Documents/repositorios/DATA_SCIENCE/woman/export/model_regressao_pregnant_risk.pkl')

# Carregar o modelo
modelo_carregado = joblib.load('/Users/mayconcyprianobatestin/Documents/repositorios/DATA_SCIENCE/woman/export/model_regressao_pregnant_risk.pkl')
